In [5]:
# Install package, compatible with API partitioning
%pip install --upgrade --quiet langchain-unstructured unstructured-client unstructured "unstructured[pdf]" python-magic

Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install python-magic-bin

ERROR: Could not find a version that satisfies the requirement python-magic-bin (from versions: none)
ERROR: No matching distribution found for python-magic-bin
Note: you may need to restart the kernel to use updated packages.


In [1]:
# from langchain_community.document_loaders import UnstructuredLoader
from langchain_unstructured import UnstructuredLoader
from langchain_community.document_loaders import PyPDFLoader
from IPython.display import display as Markdown
from tqdm.autonotebook import tqdm as notebook_tqdm

/var/folders/w2/r1ry0n8s5fd28dmlqqvd6wvm0000gn/T/ipykernel_18033/2794704736.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm as notebook_tqdm


In [2]:
local_path = "BMW_X1.pdf"

if local_path:
    loader = UnstructuredLoader(file_path=local_path)
    data = loader.load()

INFO: pikepdf C++ to Python logger bridge initialized
[nltk_data] Error loading averaged_perceptron_tagger_eng: <urlopen
[nltk_data]     error [SSL: CERTIFICATE_VERIFY_FAILED] certificate
[nltk_data]     verify failed: unable to get local issuer certificate
[nltk_data]     (_ssl.c:992)>
[nltk_data] Error loading punkt_tab: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:992)>
INFO: PDF text extraction failed, skip text extraction...


TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.

In [3]:
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [4]:
print(pages[3].page_content)

TABLE OF CONTENTS
After a vehicle software update – for example, a Remote Software Up-
grade – the Integrated Owner's Handbook for the vehicle will contain
updated information.
NOTES
Notes .....................................................................................................................  6
QUICK REFERENCE
Getting in ............................................................................................................. 22
Adjustment and operation ...............................................................................  25
On the move ....................................................................................................... 28
CONTROLS
Vehicle cockpit ...................................................................................................  36
Sensors in the vehicle ....................................................................................... 40
Vehicle operating condition ...................................................

In [6]:
#Run this 2 times to remove the index.
pages.remove(pages[3])

In [27]:
#Run this until list index goes out of bound. This will remove the entire appendix
pages.remove(pages[399])

IndexError: list index out of range

In [28]:
clean_pages = pages

In [29]:
print(clean_pages[2])

page_content='WELCOME TO BMW.
Owner's Handbook.
Congratulations on your choice of a BMW.
The better you are acquainted with your vehicle, the easier you will find it
is to operate in traffic. We therefore request:
Please read the Owner's Handbook before setting out in your new BMW.
Also use the Integrated Owner's Handbook in your vehicle. It contains
important information on how to operate your vehicle, helping you get the
most out of your BMW's technical features. It also contains information
to help keep your BMW operating safely on the road and maintain its full
resale value.
When the vehicle leaves the factory, the printed Owner's Handbook is the
most up-to-date version. After a vehicle software update – for example,
a Remote Software Upgrade – the Integrated Owner's Handbook for the
vehicle will contain updated information.
Supplementary information is provided in further on-board literature.
We wish you a safe and pleasant journey.
3
Online Edition for Part no. 01405A7CB30 - II/2

In [30]:
!ollama list

NAME                       ID              SIZE      MODIFIED     
nomic-embed-text:latest    0a109f422b47    274 MB    16 hours ago    
llava-phi3:3.8b            c7edd7b87593    2.9 GB    8 days ago      
llama3.2:latest            a80c4f17acd5    2.0 GB    2 months ago    


In [ ]:
!ollama pull nomic-embed-text

In [31]:
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [32]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

In [33]:
#First time, for creating the database

vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="nomic-embed-text"),
    collection_name="bmw-rag",
    persist_directory="./chroma_db_store",
)

In [ ]:
#For simply accessing the persisted database

vector_db = Chroma(
    embedding_function=OllamaEmbeddings(model="nomic-embed-text"),
    collection_name="bmw-rag",
    persist_directory="./chroma_db_store",
)

In [ ]:
vector_db.get()

In [34]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever
from pydantic import BaseModel

In [35]:
local_model = "llama3.2"
llm = ChatOllama(model=local_model)

In [36]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [37]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [38]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()} | prompt | llm | StrOutputParser()
)

In [39]:
chain.invoke("How to activate and deactivate the key card?")

'According to the document, to activate and deactivate the Key Card:\n\n1. Place the Key Card in the middle of the tray and push it down behind the holding clip.\n2. Follow instructions on the control display.\n\nTo deactivate the Key Card, use the following menu path via iDrive:\nmenu Apps / "VEHICLE" / "Doors and windows" / "Vehicle key" / "Key Card" / "Deactivate Key Card".'